In [1]:
import pandas as pd
from pathlib import Path
import pylatex as pl
from pylatex.utils import NoEscape
import pymc as pm
import arviz as az
import plotly.express as px
import numpy as np
import pandas as pd

from estival.model import BayesianCompartmentalModel
import estival.priors as esp
import estival.targets as est
from estival.wrappers import pymc as epm
from tbdynamics import model
from tbdynamics.inputs import fixed_parameters
from tbdynamics.utils import build_contact_matrix, round_sigfig
import plotly.graph_objects as go
#from emutools.parameters import load_param_info
#from emutools.calibration import round_sigfig
# from general_utils.doc_utils import TextElement, TableElement, FigElement, add_element_to_document, \
#     save_pyplot_add_to_doc, save_plotly_add_to_doc, compile_doc, generate_doc
from emutools.tex import StandardTexDoc

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
PROJECT_PATH = Path().resolve()
DATA_PATH = PROJECT_PATH / 'data'
SUPPLEMENT_PATH = PROJECT_PATH / 'supplement'

In [3]:
params = {
    "contact_rate": 0.009414102898074345,
    "start_population_size": 227344.75719536067,
    "cdr_adjustment": 0.6,
    "progression_multiplier": 1.1,
    "rr_infection_latent": 0.2,
    "rr_infection_recovered": 0.2,
    "infect_death_rate_unstratified": 0.21,
    "on_treatment_infect_multiplier": 0.08,
    'smear_positive_death_rate':0.364337776897486,
    'smear_negative_death_rate': 0.027588310343242016, 
    'smear_positive_self_recovery': 0.20344728302826143,
    'smear_negative_self_recovery': 0.22723824998716693,
    'rr_progression_diabetes': 4.5,
    'seed_time': 200.0,
    'seed_duration': 1.0,
    'seed_rate': 1.0

}

In [4]:
pd.options.plotting.backend = "plotly"
time_start = 1800
time_end = 2020
time_step = 0.1

doc_sections = {}
compartments = [
    "susceptible",
    "early_latent",
    "late_latent",
    "infectious",
    "on_treatment",
    "recovered",
]
infectious_compartments = [
    "infectious",
    "on_treatment",
]

latent_compartments = [
    "early_latent",
    "late_latent",
]
age_strata = [0,5,15,35,50,70] 

In [5]:
latent_compartments[0]

'early_latent'

In [6]:
app_doc = StandardTexDoc(PROJECT_PATH / 'supplement', 'supplement', "TB dynamics Supplement", 'tb')

In [7]:
matrix = build_contact_matrix()

In [8]:
tb_model = model.build_model(
    compartments,
    infectious_compartments,
    latent_compartments,
    age_strata,
    time_start,
    time_end,
    time_step,
    matrix,
    fixed_parameters,
    app_doc, 
    False
)

In [9]:
priors = [
    esp.UniformPrior("start_population_size", (1, 300000)),
    esp.UniformPrior("contact_rate", (0.0001, 0.02)),
    esp.UniformPrior("rr_infection_latent", (0.2, 0.5)),
    esp.UniformPrior("rr_infection_recovered", (0.1, 0.5)),
    esp.UniformPrior("smear_positive_death_rate", (0.335, 0.449)),
    esp.UniformPrior("smear_negative_death_rate", (0.017, 0.035)),
    esp.UniformPrior("smear_positive_self_recovery", (0.177, 0.288)),
    esp.UniformPrior("smear_negative_self_recovery", (0.073, 0.209)),
    # esp.UniformPrior("rr_progression_diabetes", (2.0, 10.0)),
    esp.UniformPrior("progression_multiplier", (0.1, 2.0)),
    # esp.UniformPrior("acf_scaleup_shape", (0.,0.1)),
    # # esp.UniformPrior("acf_inflection_time", (1980,2019)),
    # esp.UniformPrior("acf_start_asymp", (0.,1.)),
    # esp.UniformPrior("acf_end_asymp", (0.,10.)),
    # UniformPrior("cdr_adjustment", [0.6, 1.0]),
    esp.UniformPrior("infect_death_rate_dict.smear_positive", [0.335, 0.449]),
    esp.UniformPrior("infect_death_rate_dict.smear_negative", [0.017, 0.035]),
    esp.UniformPrior("self_recovery_rate_dict.smear_positive", [0.177, 0.288]),
    esp.UniformPrior("self_recovery_rate_dict.smear_negative", [0.073, 0.209]),
    esp.UniformPrior("seed_time", [1800.0, 1960.0]),
    esp.UniformPrior("seed_duration", [0.0, 5.0]),
    esp.UniformPrior("seed_rate", [1.0, 100.0]),
]
pop = pd.Series({2009: 1207100, 2019: 1194300})
notif = pd.Series({2011: 1495,2012: 1485,2013: 1369,2014:1405,2015:1642, 2016:1555, 2017:1440, 2018:1468, 2019:1417})
latent = pd.Series({2016:36})

targets = [
    est.NegativeBinomialTarget('total_population', pop, dispersion_param=2000),
    est.NormalTarget('percentage_latent', latent, stdev=5)
]
calibration_model = BayesianCompartmentalModel(tb_model, params, priors, targets)

In [10]:
with pm.Model() as pmc_model:
    start_params = {k: np.clip(v, *calibration_model.priors[k].bounds(0.99)) for k, v in params.items() if k in calibration_model.priors}
    variables = epm.use_model(calibration_model)
    map_params = pm.find_MAP(start=start_params, vars=variables, include_transformed=False)
    map_params = {k: float(v) for k, v in map_params.items()}
    print('Best calibration parameters found:')
for i_param, param in enumerate(map_params):
    print(f'   {param}: {round_sigfig(map_params[param], 4)} (within bound {priors[i_param].bounds()}')


Best calibration parameters found:
   start_population_size: 14090.0 (within bound (1.0, 300000.0)
   contact_rate: 0.02 (within bound (0.0001, 0.02)
   rr_infection_latent: 0.2215 (within bound (0.2, 0.5)
   rr_infection_recovered: 0.4259 (within bound (0.1, 0.5)
   smear_positive_death_rate: 0.4489 (within bound (0.335, 0.449)
   smear_negative_death_rate: 0.03499 (within bound (0.017, 0.035)
   smear_positive_self_recovery: 0.2878 (within bound (0.177, 0.288)
   smear_negative_self_recovery: 0.209 (within bound (0.073, 0.20900000000000002)
   progression_multiplier: 2.0 (within bound (0.1, 2.0)
   infect_death_rate_dict.smear_positive: 0.449 (within bound (0.335, 0.449)
   infect_death_rate_dict.smear_negative: 0.03499 (within bound (0.017, 0.035)
   self_recovery_rate_dict.smear_positive: 0.288 (within bound (0.177, 0.288)
   self_recovery_rate_dict.smear_negative: 0.2089 (within bound (0.073, 0.20900000000000002)
   seed_time: 1802.0 (within bound (1800.0, 1960.0)
   seed_duratio

In [11]:
params.update(map_params)
tb_model.run(params)
derived_df_0 = tb_model.get_derived_outputs_df()
plots = {"total_population": {
      "title": "Population size",
      "output_key": "total_population",
      "times": [2009.0, 2019.0],
      "values": [1207100, 1194300],
      "quantiles": [0.025, 0.25, 0.5, 0.75, 0.975]
    },
     "notifications": {
      "title": "Notifications",
      "output_key": "notifications",
      "times": [2011.0, 2012.0, 2013.0, 2014.0, 2015.0, 2016.0, 2017.0, 2018.0, 2019.0],
      "values": [1495, 1485, 1369, 1405, 1642, 1555, 1440, 1468, 1417],
      "quantiles": [0.025, 0.25, 0.5, 0.75, 0.975]
    },
    "percentage_latent": {
      "title": "Percentage Latent",
      "output_key": "percentage_latent",
      "times": [2016.0],
      "values": [30.8],
      "quantiles": [0.025, 0.25, 0.5, 0.75, 0.975]
    },
    
    }

In [12]:
fig2_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="total_population",
)
fig2_2 = px.scatter(x= plots['total_population']['times'], y = plots['total_population']['values'])
fig2_2.update_traces(marker=dict(color="red"))
fig2_3 = go.Figure(
    data=fig2_1.data + fig2_2.data,
)
fig2_3.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Population"
)
fig2_3.show()


In [13]:
pop = derived_df_0[[f'total_populationXage_{i}' for i in age_strata]] 

In [14]:
pop.plot.area()

In [15]:
pop.plot.area(log_y=True)

In [16]:
derived_df_0

,infectious_population_size,latent_population_size,percentage_latent,prevalence_infectious,total_population,total_populationXage_0,total_populationXage_15,total_populationXage_35,total_populationXage_5,total_populationXage_50,total_populationXage_70
1800.0,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,1.408607e+04,2347.678152,2347.678152,2347.678152,2347.678152,2347.678152,2347.678152
1800.1,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,1.410143e+04,2348.150055,2357.614301,2341.232200,2370.005677,2346.176377,2338.254909
1800.2,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,1.411694e+04,2348.671580,2367.714469,2334.886021,2392.111165,2344.643038,2328.908824
1800.3,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,1.413257e+04,2349.242107,2377.975511,2328.639651,2413.997905,2343.079028,2319.639047
1800.4,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,1.414835e+04,2349.861025,2388.394336,2322.493108,2435.669131,2341.485235,2310.444745
...,...,...,...,...,...,...,...,...,...,...,...
2019.6,3.099033e-07,0.000052,4.175707e-09,2.468944e-08,1.255206e+06,101232.055584,372689.924916,243348.103290,198473.612562,217528.291464,121933.992798
2019.7,3.084947e-07,0.000052,4.154140e-09,2.456004e-08,1.256084e+06,101290.392404,372749.121541,243519.513011,198498.449620,217847.001825,122179.487819
2019.8,3.070892e-07,0.000052,4.132685e-09,2.443107e-08,1.256961e+06,101349.021318,372808.265421,243690.030163,198524.201391,218164.944823,122424.879400
2019.9,3.056879e-07,0.000052,4.111343e-09,2.430264e-08,1.257838e+06,101407.935388,372867.365933,243859.660688,198550.864458,218482.119127,122670.164427
